In [1]:
import os
import pandas as pd
import geopandas as gpd
import folium
from urbantrips.utils import utils
from urbantrips.kpi import overlapping as ovl
from urbantrips.viz import overlapping as ovl_viz

import itertools

URBANTRIPS_PATH = "/home/pipe/proyectos/urbantrips/ciudades/amba_2024/UrbanTrips"

os.chdir(URBANTRIPS_PATH)

configs = utils.leer_configs_generales()
conn_data = utils.iniciar_conexion_db(tipo='data')
conn_insumos = utils.iniciar_conexion_db(tipo='insumos')

In [2]:
metadata_lineas = pd.read_sql("select id_linea,nombre_linea, modo from metadata_lineas;", conn_insumos)
metadata_lineas.sample(3)


,id_linea,nombre_linea,modo
349,1253,BSAS_LINEA_140,autobus
102,392,LINEA 410,autobus
398,172,BSAS_LINEA_177,autobus


In [17]:
metadata_lineas[metadata_lineas.nombre_linea.str.contains("50")]

,id_linea,nombre_linea,modo
2,35,LINEA 50,autobus
7,45,BSAS_LINEA_150,autobus
96,363,BSAS_LINEA_503C,autobus
104,394,BSAS_LINEA_502,autobus
124,211,LINEA 501 A,autobus
128,358,BSAS_LINEA_501F,autobus
129,352,BSAS_LINEA_500C,autobus
130,340,BSAS_LINEA_502B,autobus
134,362,LINEA 503 B,autobus
136,414,BSAS_LINEA_508C,autobus


In [18]:
# Set configurations for day, h3 resolution and route geoms
day = "2024-05-14"

h3_res_comp = 7 # 8

# id linea 41 para la linea 12
# id linea 146 para la linea 39

# id linea 39 para la LINEA 6	
# id linea 35 para la LINEA 50	
comp_line_id = 39
base_line_id = 35



In [23]:
use_branches = configs['lineas_contienen_ramales']
#use_branches = False
if use_branches:
    metadata = pd.read_sql(f"select id_linea,id_ramal from metadata_ramales where id_linea in ({base_line_id},{comp_line_id})",
                            conn_insumos,dtype={'id_linea': int, 'id_ramal': int})
    route_type = 'branches'
    # get al combinations of route ids form branches
    route_id_combinations = list(itertools.combinations(metadata['id_ramal'], 2))
    base_route_id_combinations = list(itertools.combinations(metadata.loc[metadata.id_linea == base_line_id,'id_ramal'], 2))
    comp_line_id_combinations = list(itertools.combinations(metadata.loc[metadata.id_linea == comp_line_id,'id_ramal'], 2))
    route_id_combinations = [combination for combination in route_id_combinations if ((combination not in base_route_id_combinations) and (combination not in comp_line_id_combinations))]
    metadata_branches = pd.read_sql(f"select * from metadata_ramales where id_linea in ({base_line_id},{comp_line_id})",
                                conn_insumos,dtype={'id_linea': int, 'id_ramal': int})

else:
    #metadata = pd.read_sql(f"select id_line from metadata_lineas where id_linea in ({base_line_id},{comp_line_id})", conn_insumos)    
    route_type = 'lines'
    route_id_combinations = [(base_line_id, comp_line_id)]



In [24]:
metadata_branches

,id_ramal,id_linea,nombre_ramal,modo,empresa,descripcion
0,49,35,050A,autobus,NUDO S.A.,None
1,449,39,006B,autobus,NUDO S.A.,None


In [25]:
route_id_combinations

[(49, 449)]

In [26]:
route_id_combination = route_id_combinations[0] 
route_id_combination


(49, 449)

In [27]:
if use_branches:
    if route_id_combination[0] in metadata.loc[metadata.id_linea == base_line_id,'id_ramal'].values:
        base_route_id = route_id_combination[0]
        comp_route_id = route_id_combination[1]
    else:
        base_route_id = route_id_combination[1]
        comp_route_id = route_id_combination[0]
else:
    base_route_id,comp_route_id = route_id_combination

In [28]:
print("base_line_id",base_line_id)
print("base_route_id",base_route_id)
print("comp_line_id",comp_line_id)
print("comp_route_id",comp_route_id)


base_line_id 35
base_route_id 49
comp_line_id 39
comp_route_id 449


In [29]:
overlapping_dict = ovl.compute_supply_overlapping(day, base_route_id,comp_route_id,route_type,h3_res_comp)  
base_gdf = overlapping_dict["base"]["h3"]
base_route_gdf = overlapping_dict["base"]["line"]
comp_gdf = overlapping_dict["comp"]["h3"]
comp_route_gdf = overlapping_dict["comp"]["line"]


El 46.7 % del recorrido del ramal base 050A se superpone con el del ramal de comparación 006B
Por otro lado 70.0 % del recorrido del ramal 006B se superpone con el del ramal 050A


In [30]:
f = ovl_viz.plot_interactive_supply_overlapping(overlapping_dict)
f

In [31]:
# demanda
base_demand,comp_demand = ovl.compute_demand_overlapping(base_line_id,comp_line_id,day,
                                                         base_route_id,comp_route_id,
                                                         base_gdf,comp_gdf)

Obteniendo datos de etapas
Obteniendo datos de etapas
La demanda total para la linea LINEA 50 (id 35) que podria recorrer este ramal 050A (id 49) es: 26161 etapas
De las cuales el 29.0 % comparte OD con la linea LINEA 6 (id 39) ramal 006B (id 449)
La demanda total para la linea LINEA 6 (id 39) que podria recorrer este ramal 006B (id 449) es: 16869 etapas
De las cuales el 37.4 % comparte OD con la linea LINEA 50 (id 35) ramal 050A (id 49)


In [32]:
fig = ovl_viz.plot_interactive_demand_overlapping(base_demand, comp_demand, overlapping_dict)
fig